In [15]:
import os.path
import pandapower.networks as pn
import pandapower as pp
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from numpy import flatnonzero as find, ones, zeros, Inf, pi, exp, conj, r_, arange, array
from pypower_.makeYbus import makeYbus
from pypower_.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower_.idx_gen import GEN_BUS, PG, QG
from pypower_.idx_cost import MODEL, PW_LINEAR, NCOST, POLYNOMIAL
from pypower_.totcost import totcost
from pypower_.makeSbus import makeSbus
from pypower_.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
import cvxpy as cp
import numpy as np
from scipy.sparse import csr_matrix as sparse
from pypower_.idx_bus import PD, QD
from pypower_.idx_gen import GEN_BUS, PG, QG, GEN_STATUS

In [16]:
import sys
print(sys.path)
print(os.path)
path_current = '/home/ubuntu-h/PycharmProjects/scientificProject'
if path_current not in sys.path:
    sys.path.insert(1, '/home/ubuntu-h/PycharmProjects/scientificProject')

['/snap/pycharm-professional/319/plugins/python/helpers-pro/jupyter_debug', '/snap/pycharm-professional/319/plugins/python/helpers/pydev', '/home/ubuntu-h/PycharmProjects/scientificProject/notebooks', '/home/ubuntu-h/PycharmProjects/scientificProject', '/home/ubuntu-h/PycharmProjects/scientificProject/src/pysettrie', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python310.zip', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/lib-dynload', '', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/site-packages', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg']
<module 'posixpath' from '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/posixpath.py'>


In [17]:
import pandapower as pp
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
# bus4 = pp.create_bus(net, vn_kv=110.)
# bus5 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
# pp.create_line(net, bus3, bus4, length_km=50., std_type="149-AL1/24-ST1A 110.0")
# pp.create_line(net, bus4, bus2, length_km=40., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
pp.create_load(net, bus1, p_mw=30., controllable=False)
# pp.create_load(net, bus3, p_mw=70., controllable=False)
# pp.create_load(net, bus4, p_mw=25., controllable=False)

# create generators
eg = pp.create_ext_grid(net, bus3, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus2, p_mw=80, min_p_mw=0, max_p_mw=50, vm_pu=1.00, controllable=True)
# g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.00, controllable=True)


costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

![xx -w50](grid2GUROBI.png)

In [18]:
# net.bus["min_vm_pu"] = 0.98
# net.bus["max_vm_pu"] = 1.02
net.line["max_loading_percent"] = 100
# net.sn_mva = 100
# net = pp.networks.case14()
# net.gen['vm_pu'] = 1.0
# net.bus["min_vm_pu"] = 0.95
# net.bus["max_vm_pu"] = 1.05
# net.poly_cost[:,3] = 0
# net.poly_cost['cp2_eur_per_mw2'] = 0
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,10.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,10.0,0.0,0.0,0.0,0.0


In [19]:
om,ppopt = pp.runopp(net, delta=1e-16)   # solve the OPF problem

In [20]:
# net = pp.networks.case57()
print(net.res_gen)    # output the results of two generators
net.gen

        p_mw    q_mvar  va_degree     vm_pu
0  30.972595  0.434275   -0.12696  1.054942


,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,min_p_mw,max_p_mw,power_station_trafo
0,None,1,80.0,1.0,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,50.0,NaN


In [21]:
print(net.res_ext_grid)
net.ext_grid['vm_pu']

      p_mw    q_mvar
0  0.00008 -3.470302


0    1.05
Name: vm_pu, dtype: float64

In [22]:
# the total cost of generation
net.res_cost

309.7275380408373

In [23]:
# the key variable, which represents the information of the OPF problem
om.get_ppc()

{'baseMVA': 1,
 'version': 2,
 'bus': array([[ 0.00000000e+00,  1.00000000e+00,  3.00000000e+01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.02244822e+00, -3.98223525e+00,
          1.10000000e+02,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  1.06688085e+01,  1.79874386e-02,
          0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  2.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.05494243e+00, -1.26959649e-01,
          1.10000000e+02,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  1.00000162e+01,  6.93307350e-22,
          0.00000000e+00,  0.00000000e+00],
        [ 2.00000000e+00,  3.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.05000000e+00,  0.00000000e+00,
          1.10000000e+02,  1.00000000e+00,  1.05000000e+00,
          1.05000000e+00,  9.979068

In [24]:
## unpack data
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]
vv, _, _, _ = om.get_idx()

## problem dimensions
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches
ny = om.getN('var', 'y')   ## number of piece-wise linear costs, 'y' is a part of p-w linear costs

## bounds on optimization vars
x0, xmin, xmax = om.getv()

In [25]:
## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)
Ybus.todense()

/home/ubuntu-h/PycharmProjects/scientificProject/pypower_/makeYbus.py:49: ComplexWarning: Casting complex values to real discards the imaginary part
  tap[i] = branch[i, TAP]  ## assign non-zero tap ratios
/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/site-packages/scipy/sparse/_coo.py:160: ComplexWarning: Casting complex values to real discards the imaginary part
  self.col = np.array(col, copy=copy, dtype=idx_dtype)


matrix([[ 162.99668368-343.31337094j, -162.99668368+344.47752737j,
            0.          +0.j        ],
        [-162.99668368+344.47752737j,  325.99336737-686.62674189j,
         -162.99668368+344.47752737j],
        [   0.          +0.j        , -162.99668368+344.47752737j,
          162.99668368-343.31337094j]])

In [26]:
## Set the lower and upper bound for all variables
ll, uu = xmin.copy(), xmax.copy()
# print(f'll : {ll}; uu : {uu}')
ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
uu[xmax ==  Inf] =  1e10
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
# print(f"Va_refs: {Va_refs}")
ll[vv["i1"]["Va"]:vv["iN"]["Va"]] = -np.ones_like(bus[:, VA]) * (pi / 2) # Va lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]] = np.ones_like(bus[:, VA]) * (pi / 2) # Va upper bound 赋值
## deal with the Va_refs
ll[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs  # Va_refs lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs # Va_refs upper bound 赋值
## deal with reactive power, whose ll and uu is 1e9
ll[ll<-1e4] = -100
uu[uu>1e4] = 100
v_max = uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]][-1]
print(f'll"{ll};\n uu:{uu}')

ll"[-1.57079633e+00 -1.57079633e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.05000000e+00 -1.00000000e-16 -1.00000000e-16
 -1.00000000e+02 -1.00000000e+02];
 uu:[   1.57079633    1.57079633    0.            2.            2.
    1.05       1000.           50.          100.          100.        ]


In [27]:
x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]]

array([1.  , 1.  , 1.05])

In [28]:
uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]]

array([2.  , 2.  , 1.05])

In [29]:
bus[:, VM] # 赋值

array([1.02244822, 1.05494243, 1.05      ])

In [30]:
x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]] = bus[:, VM] # 赋值

In [31]:
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
x0[vv["i1"]["Va"]:vv["iN"]["Va"]] = bus[:, VA] * (pi / 180) # 赋值

In [32]:
## 赋值
x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]] = gen[:,PG]
x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]] = gen[:,QG]


### 不能处理piecewise linear 的cost, 因为没有引入那个函数，（大约在这个位置）


### 处理cost fun

In [33]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

##----- evaluate objective function -----
## polynomial cost of P and Q
# use totcost only on polynomial cost in the minimization problem formulation, pwl cost is the sum of the y variables.
ipol = find(gencost[:, MODEL] == POLYNOMIAL)   ## poly MW and MVAr costs
xx = r_[ Pg, Qg ] * baseMVA
if len(ipol)>0:
    f = sum( totcost(gencost[ipol, :], xx[ipol]) )  ## cost of poly P or Q
else:
    f = 0

In [34]:
Pg

array([7.95042360e-05, 3.09725948e+01])

In [35]:
xx[ipol]

array([7.95042360e-05, 3.09725948e+01])

In [36]:
First_Or_Con = 4
print(f"{gencost[ipol, First_Or_Con], gencost[:, First_Or_Con]}")

(array([20., 10.]), array([20., 10.]))


In [37]:
gencost

array([[ 2.,  0.,  0.,  2., 20.,  0.],
       [ 2.,  0.,  0.,  2., 10.,  0.]])

### 处理约束部分

In [38]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

In [39]:
## put Pg & Qg back in gen
gen[:, PG] = Pg * baseMVA  ## active generation in MW
gen[:, QG] = Qg * baseMVA  ## reactive generation in MVAr

In [40]:

on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
gbus = gen[on, GEN_BUS]                   ## what buses are they at?

## form net complex bus power injection vector
nb = bus.shape[0]
ngon = on.shape[0]
## connection matrix, element i, j is 1 if gen on(j) at bus i is ON
Cg = sparse((ones(ngon), (gbus, range(ngon))), (nb, ngon))

## power injected by gens plus power injected by loads converted to p.u.
Sbus = ( Cg * (gen[on, PG] + 1j * gen[on, QG]) - (bus[:, PD] + 1j * bus[:, QD]) ) / baseMVA
# Sbus = makeSbus(baseMVA, bus, gen) ## net injected power in p.u.

In [41]:
Sbus

array([-3.00000000e+01+0.j        ,  3.09725948e+01+0.43427464j,
        7.95042360e-05-3.47030236j])

In [42]:
## reconstruct V
Va = x0[vv["i1"]["Va"]:vv["iN"]["Va"]]
Vm = x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]]
V = Vm * exp(1j * Va)
V

array([1.01997965-0.07100614j, 1.05493984-0.00233761j,
       1.05      +0.j        ])

In [43]:
Va

array([-0.06950312, -0.00221586,  0.        ])

In [44]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.022448,-3.982235,30.000000,0.000000,10.668808,1.798744e-02
1,1.054942,-0.126960,-30.972595,-0.434275,10.000016,6.933073e-22
2,1.050000,0.000000,-0.000080,3.470302,9.979068,-5.531631e-21


In [45]:
## evaluate power flow equations
mis = V * conj(Ybus * V) - Sbus

In [46]:
ppc['branch']

array([[ 1.00000000e+00+0.j,  2.00000000e+00+0.j,  1.12231405e-03+0.j,
         2.37190083e-03+0.j,  2.32831286e+00+0.j,  8.95470268e+01+0.j,
         2.50000000e+02+0.j,  2.50000000e+02+0.j,  1.00000000e+00+0.j,
         0.00000000e+00+0.j,  1.00000000e+00+0.j, -3.60000000e+02+0.j,
         3.60000000e+02+0.j,  4.78862237e-03+0.j,  9.01514410e-01+0.j,
         7.95039119e-05+0.j, -3.47030236e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  1.00000000e+00+0.j,  1.12231405e-03+0.j,
         2.37190083e-03+0.j,  2.32831286e+00+0.j,  8.95470268e+01+0.j,
         2.50000000e+02+0.j,  2.50000000e+02+0.j,  1.00000000e+00+0.j,
         0.00000000e+00+0.j,  1.00000000e+00+0.j, -3.60000000e+02+0.j,
         3.60000000e+02+0.j, -3.00000000e+01+0.j, -2.07389661e-11+0.j,
         3.09678062e+01+0.j, -4.67239774e-01+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.

In [47]:
branch[:, RATE_A]

array([89.54702675+0.j, 89.54702675+0.j])

In [48]:
 ## find branches with flow limits
il = find((branch[:, RATE_A] != 0) & (branch[:, RATE_A] < 1e10))
nl2 = len(il)           ## number of constrained lines

In [49]:
il

array([0, 1])

In [60]:
# Parameters
# obtain the complex load
Pd = bus[:, PD]
Qd = bus[:, QD]
# slack_v = net.ext_grid['vm_pu']


def optimize_OPF(p_load):
    Pd = p_load
    Qd = 0

    lb_update = -1e4

    # obtain the range of voltage; the range is from the setting
    V_range = 0.1
    V_ll,V_uu = 1.0-V_range, 1.0+V_range

    # obtain the power range
    # Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]*baseMVA
    # Qg_lower, Qg_upper = ll[vv["i1"]["Qg"]:vv["iN"]["Qg"]], uu[vv["i1"]["Qg"]:vv["iN"]["Qg"]]*baseMVA

    # the dimension of the OPF
    # nb = bus.shape[0]          ## number of buses
    # nl = branch.shape[0]       ## number of branches

    # ineq_idx = 0

    # create the model; set the model type: NonConvex.
    model = gp.Model("PowerGeneration");model.setParam('NonConvex', 2)

    # creat variables for the model; the numbers in the following codes should be replaced by parameters
    V_re = model.addMVar(3, lb=[-1.05,-1.05,-1.05], ub=[1.05,1.05,1.05], vtype=GRB.CONTINUOUS, name='V real')
    V_im = model.addMVar(3, lb=[-1.05,-1.05,-1.05],ub=[1.05,1.05,1.05], vtype=GRB.CONTINUOUS, name='V imag')
    Pg = model.addMVar(2, lb=0, ub=[105,105], vtype=GRB.CONTINUOUS, name='Pg')
    Qg = model.addMVar(2, lb=[0,0], ub=[105,105], vtype=GRB.CONTINUOUS, name='Qg')
    temp1 = model.addMVar(3, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 1')
    temp2 = model.addMVar(3, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 2')
    Pg_net = model.addMVar(3, lb=lb_update, vtype=GRB.CONTINUOUS, name='Pg net')  # the number of buses, too
    Qg_net = model.addMVar(3, lb=lb_update, vtype=GRB.CONTINUOUS, name='Qg net')

    # Line_temp_1 = model.addMVar(2, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 1')    # the shape should be line number
    # Line_temp_2 = model.addMVar(2, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 2')    # the shape should be line number
    # Line_temp_3 = model.addMVar(2, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 3')    # the shape should be line number
    # Line_temp_4 = model.addMVar(2, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 4')    # the shape should be line number

    # the prameters of branches
    G = np.real(Ybus)
    B = np.imag(Ybus)


    # Constraints; insert the formulas
    Temp_c1 = model.addConstr(G@V_re - B@V_im == temp1, 'Temp Constr 1')
    Temp_c2 = model.addConstr(G @ V_im + B @ V_re == temp2, 'Temp Constr 2')
    Pg_net_c = model.addConstr(Cg @ Pg - Pd == Pg_net, 'Pg net Constr')
    Qg_net_c = model.addConstr(Cg @ Qg - Qd == Qg_net, 'Qg net Constr')
    # print(f'Pd:{Pd}\n Qd:{Qd}')
    CVL_CPL = model.addConstrs((V_re[i] * temp1[i] *baseMVA+ V_im[i] * temp2[i]*baseMVA == Pg_net[i]  for i in range(nb)), 'Cvl P')  # break into different parts
    CVL_CPL_Q = model.addConstrs((V_im[i] * temp1[i] *baseMVA - V_re[i] * temp2[i]*baseMVA == Qg_net[i] for i in range(nb)), 'Cvl Q')

    # the constraints of voltages on all buses; set the constraint for slack bus, then give the upper and lower constraints for other buses

    # the upper&lower constraints for buses except for the slack bus
    V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 <= V_uu**2 for i in range(1,nb-ineq_idx)), 'V upper bound')  # 对 range 起始值的处理，是处理树状分层搜索的突破口；有一"减"，对应的有一"加"。
    # if ineq_idx != 0:
    #     V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 == V_uu**2 for i in range(ineq_idx,nb)), 'V upper bound')
    #
    V_lower = model.addConstrs((V_re[i]**2 + V_im[i]**2 >= V_ll**2 for i in range(1,nb)), 'V lower bound')

    # # Yf * W * Yf; the number is equal to that of the lines/branches
    # Line_temp_c_1 = model.addConstr(G_yf@V_re - B_yf@V_im == Line_temp_1, 'Branch Con 1')
    # Line_temp_c_2 = model.addConstr(G_yf @ V_im + B_yf @ V_re == Line_temp_2, 'Branch Con 2')
    # Line_temp_c_3 = model.addConstr(G_yt@V_re - B_yt@V_im == Line_temp_3, 'Branch Con 3')
    # Line_temp_c_4 = model.addConstr(G_yt @ V_im + B_yt @ V_re == Line_temp_4, 'Branch Con 4')
    # # 下式左侧是否应该乘上baseMVA
    # Line_c_1 = model.addConstrs((Line_temp_1[i]**2 + Line_temp_2[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 1') # p.u.
    # Line_c_2 = model.addConstrs((Line_temp_3[i]**2 + Line_temp_4[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 2') # p.u.

    # set the objective of the model
    print('ipol:', ipol)
    # The "gencost" changes with different cases. Keep in Mind!
    # model.setObjective(gencost[ipol, First_Or_Con]@Pg**2+gencost[ipol, First_Or_Con+1]@Pg, GRB.MINIMIZE)
    model.setObjective(np.array([1, 1.5])@Pg, GRB.MINIMIZE)

    # calculate the optimization
    model.optimize()

    Pg_star, Qg_star = Pg.X, Qg.X
    optimized_obj = np.array([1, 1.5])@Pg_star

    print(f'Pg: {Pg_star}; Qg: {Qg_star}; opt obj: {optimized_obj}')

optimize_OPF(p_load=30)

Set parameter NonConvex to value 2
ipol: [0 1]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-10600KF CPU @ 4.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 22 columns and 44 nonzeros
Model fingerprint: 0x4a87b7bb
Model has 10 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [3e+01, 3e+01]
  QRHS range       [8e-01, 1e+00]
Presolve removed 6 rows and 6 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 66 rows, 32 columns, 202 nonzeros
Presolved model has 2 quadratic constraint(s)
Presolved model has 16 bilinear constraint(s)
Variable types: 32 continuous, 0 integer (0 binary)

Root

In [56]:
i = 0
for p in range(50):
    i+=1
    p_load = p*0.0005
    print(f'is: {i}')
    optimize_OPF(p_load)

array([32.18896129,  0.        ])

In [57]:
Pd

array([30.,  0.,  0.])

In [58]:
Qd

array([0., 0., 0.])

In [41]:
llv = ll[vv["i1"]["Vm"]:vv["iN"]["Vm"]]**2
uuv = uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]]**2
uuv

array([1.1025, 1.0404, 1.0404, 1.0404])

In [42]:
print(Ybus)

  (0, 0)	(162.99668368338897-343.3133709444148j)
  (1, 0)	(-162.99668368338897+344.47752737211067j)
  (0, 1)	(-162.99668368338897+344.47752737211067j)
  (3, 1)	(-285.24419644593075+602.8356729011938j)
  (1, 1)	(611.2375638127087-1288.7971825456257j)
  (2, 1)	(-162.99668368338897+344.47752737211067j)
  (2, 2)	(391.19204084013353-824.7503689598727j)
  (1, 2)	(-162.99668368338897+344.47752737211067j)
  (3, 2)	(-228.19535715674456+482.268538320955j)
  (3, 3)	(513.4395536026753-1083.607438672254j)
  (2, 3)	(-228.19535715674456+482.268538320955j)
  (1, 3)	(-285.24419644593075+602.8356729011938j)


In [43]:
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]
Pg_upper

array([1000.,   80.,  100.])

In [44]:
Pd = bus[:, PD]
Pd

array([ 0., 60., 70., 25.])

In [45]:
baseMVA

1

In [46]:
# Parameters
# obtain the complex load
Pd = bus[:, PD]
Qd = bus[:, QD]
slack_v = net.ext_grid['vm_pu']

lb_update = -1e4

# obtain the range of voltage; the range is from the setting
V_range = v_max - 1
V_ll,V_uu = 1.0-V_range, 1.0+V_range

# obtain the power range
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]*baseMVA
Qg_lower, Qg_upper = ll[vv["i1"]["Qg"]:vv["iN"]["Qg"]], uu[vv["i1"]["Qg"]:vv["iN"]["Qg"]]*baseMVA

# the dimension of the OPF
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches

ineq_idx = 0

# create the model; set the model type: NonConvex.
model = gp.Model("PowerGeneration");model.setParam('NonConvex', 2)

# creat variables for the model; the numbers in the following codes should be replaced by parameters
V_re = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V real')
V_im = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V imag')
Pg = model.addMVar(ngon, lb=Pg_lower, ub=Pg_upper, vtype=GRB.CONTINUOUS, name='Pg')
Qg = model.addMVar(ngon, lb=Qg_lower, ub=Qg_upper, vtype=GRB.CONTINUOUS, name='Qg')
temp1 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 1')
temp2 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 2')
Pg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Pg net')  # the number of buses, too
Qg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Qg net')
Line_temp_1 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 1')    # the shape should be line number
Line_temp_2 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 2')    # the shape should be line number
Line_temp_3 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 3')    # the shape should be line number
Line_temp_4 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 4')    # the shape should be line number

# the prameters of branches
G = np.real(Ybus)
B = np.imag(Ybus)
# the parameters for currents in branches
G_yf, B_yf = np.real(Yf), np.imag(Yf)
G_yt, B_yt = np.real(Yt), np.imag(Yt)

# Constraints; insert the formulas
Temp_c1 = model.addConstr(G@V_re - B@V_im == temp1, 'Temp Constr 1')
Temp_c2 = model.addConstr(G @ V_im + B @ V_re == temp2, 'Temp Constr 2')
Pg_net_c = model.addConstr(Cg @ Pg - Pd == Pg_net, 'Pg net Constr')
Qg_net_c = model.addConstr(Cg @ Qg - Qd == Qg_net, 'Qg net Constr')
# print(f'Pd:{Pd}\n Qd:{Qd}')
CVL_CPL = model.addConstrs((V_re[i] * temp1[i] *baseMVA+ V_im[i] * temp2[i]*baseMVA == Pg_net[i]  for i in range(nb)), 'Cvl P')  # break into different parts
CVL_CPL_Q = model.addConstrs((V_im[i] * temp1[i] *baseMVA - V_re[i] * temp2[i]*baseMVA == Qg_net[i] for i in range(nb)), 'Cvl Q')

# the constraints of voltages on all buses; set the constraint for slack bus, then give the upper and lower constraints for other buses
V_slack = model.addConstr(V_re[0]==slack_v, 'V slack real')
V_slack_a = model.addConstr(V_im[0]==0, 'V slack imag')
# the upper&lower constraints for buses except for the slack bus
V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 <= V_uu**2 for i in range(1,nb-ineq_idx)), 'V upper bound')  # 对 range 起始值的处理，是处理树状分层搜索的突破口；有一"减"，对应的有一"加"。
# if ineq_idx != 0:
#     V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 == V_uu**2 for i in range(ineq_idx,nb)), 'V upper bound')
#
V_lower = model.addConstrs((V_re[i]**2 + V_im[i]**2 >= V_ll**2 for i in range(1,nb)), 'V lower bound')

# Yf * W * Yf; the number is equal to that of the lines/branches
Line_temp_c_1 = model.addConstr(G_yf@V_re - B_yf@V_im == Line_temp_1, 'Branch Con 1')
Line_temp_c_2 = model.addConstr(G_yf @ V_im + B_yf @ V_re == Line_temp_2, 'Branch Con 2')
Line_temp_c_3 = model.addConstr(G_yt@V_re - B_yt@V_im == Line_temp_3, 'Branch Con 3')
Line_temp_c_4 = model.addConstr(G_yt @ V_im + B_yt @ V_re == Line_temp_4, 'Branch Con 4')
# 下式左侧是否应该乘上baseMVA
Line_c_1 = model.addConstrs((Line_temp_1[i]**2 + Line_temp_2[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 1') # p.u.
Line_c_2 = model.addConstrs((Line_temp_3[i]**2 + Line_temp_4[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 2') # p.u.

# set the objective of the model
print('ipol:', ipol)
# The "gencost" changes with different cases. Keep in Mind!
# model.setObjective(gencost[ipol, First_Or_Con]@Pg**2+gencost[ipol, First_Or_Con+1]@Pg, GRB.MINIMIZE)
model.setObjective(gencost[ipol, First_Or_Con]@Pg, GRB.MINIMIZE)

# calculate the optimization
# model.optimize()
model.X[:14]

Pg_star, Qg_star = Pg.X, Qg.X
optimized_obj = gencost[ipol, First_Or_Con]@Pg_star


Set parameter NonConvex to value 2
ipol: [0 1 2]


GurobiError: Index 8 out of range for attribute 'X'

In [ ]:
len(model.getVars())
len(model.getConstrs())

In [ ]:
print(Pg)

In [ ]:
net.res_bus

In [ ]:
m = gp.Model("model")
# x = m.addMVar(shape=(10), name="x")
y = m.addMVar(shape=(5),lb=np.array([-1,-2,-3,-4,-5]), name="y")

A = np.ones(5)

m.addConstrs((y[i] >=np.array([-1,-2,-3,-4,-5])[i] for i in range(5)), name="const")
m.setObjective(A@Y_.real)
m.optimize()

In [ ]:
m.X

In [ ]:
for v in m.getVars():
    print(v.varName, v.x)

In [ ]:
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

print(f'A ;{A};\n b : {b}')

In [ ]:
# Define and solve the CVXPY problem.
# Create a symmetric matrix variable
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve()
# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)